In [11]:
import sqlite3
import os
import pandas as pd
import numpy as np
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

connection = sqlite3.connect('instance/db.sqlite')
cur = connection.cursor()
PATH = 'project/static/images/'

In [14]:
data = cur.execute('''SELECT id, img_path, clothes, color FROM photo''').fetchall()
columns = ['id', 'img_path', 'clothes', 'color']
data = pd.DataFrame(columns=columns, data=data)
data['clothes'] = data.clothes.apply(lambda x: x.split(','))
for i in range(21):
    data[f'{i}_clothes'] = data.clothes.apply(lambda x: int(i in x))
colors = list(map(str.strip, open('img_proessing/colors.txt').readlines()))
for i in colors:
    data[i+'_color'] =  data.color.apply(lambda x: int(i == x))

In [15]:
from sklearn.metrics.pairwise import cosine_distances
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image, ImageShow

resnet = models.resnet152(pretrained=True)
for param in resnet.parameters():
    param.requires_grad = False
resnet = torch.nn.Sequential(*(list(resnet.children())[:-1]))


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
])

def process_image(image_path):
    image = Image.open(PATH + image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)
    return image


def get_resnet_embedding(image_path):
    image = process_image(image_path)
    with torch.no_grad():
        resnet.eval()
        output = resnet(image)
    return output.squeeze().numpy() 


In [16]:
data['embedings'] = data['img_path'].apply(get_resnet_embedding)
data.embedings = data.embedings.apply(lambda x: [x.tolist()])
emb = data.set_index('id')['embedings']

In [ ]:
columns = data.id.unique()
dist_df = pd.DataFrame(index=columns, columns=columns)
for i in range(len(columns)):
    for j in range(i, len(columns)):
        dist_df[columns[i]][columns[j]] = cosine_distances(emb[columns[i]], emb[columns[j]])[0][0]
        dist_df[columns[j]][columns[i]] = dist_df[columns[i]][columns[j]]
dist_df

In [35]:
dist_df.to_csv('dist.csv', sep=';', index_label='img_id')

In [13]:
connection.executescript('''DELETE FROM interaction''')
connection.commit()
# connection.close()